Revan Minnam

gini_index: The index of Gini Coefficent, which is a measure that shows the income inequality or the wealth inequality within a country. A higher Gini index indicates greater inequality, with high-income individuals receiving much larger percentages of the total income of the population.

corruption_perceptions_index: an index which ranks countries based on their level of corruption within a country. The CPI generally defines corruption as an "abuse of entrusted power for private gain"

freedom_house: I am assuming Freedom House has assessed the condition of political rights and civil liberties around the world. This is by the Freedom House Non-profit Organization (funded by the U.S.)

HDI: Human Development Index (HDI) is statistic composite index of life expectancy, education (mean years of schooling completed and expected years of schooling upon entering the education system), and per capita income indicators, which are used to rank countries into four tiers of human development. According to The Economic Times, A country scores a higher HDI when the lifespan is higher, the education level is higher, and the gross national income GNI (PPP) per capita is higher.

press_freedom: The Press Freedom Index is an annual ranking of countries based on an assessment of the countries' press freedom records in the previous year. It intends to reflect the degree of freedom that journalists, news organisations, and netizens have in each country, and the efforts made by authorities to respect this freedom. 

democracy_economist: According to eiu.com, the Democracy Index is based on five categories: electoral process and pluralism, civil liberties, the functioning of government, political participation, and political culture. Each country is classified based on their score: full democracy, flawed democracy, hybrid regime or authoritarian regime.

populism: Populism refers to a range of political stances. This is essentially a measure based on the political spectrum. i.e left-wing to right-wing

effective_coverage_of_health_services_index: Measuring universal health coverage based on an index of effective coverage of health services

trust_in_news_media: An index measuring the people's trust in their own media/news sites.

trust_in_government: An index measuring the people's trust in their own goverment.

trust_in_science:  An index measuring the people's trust in science.

colonized: A binary representation to whether or not a country/nation is colonized

In [180]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sys
from scipy import interpolate
from scipy.spatial import ConvexHull
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

random.seed(265)

def main():
    df = pd.read_excel("/Users/revan/Downloads/country_information.xlsx")
    df1 = problem1(df.copy())
    df2 = problem2(df1)
    df3 = problem3(df)
    df3['country'] = df['country']
    problem4(df3)

def problem1(df):
    df['country'] = pd.Categorical(df['country']).codes
    df = NormalizeData(df)
    return df

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def problem2(data):
    df = data[list(set(data.columns)-set(['country']))]
    x = df.values
    scaler = MinMaxScaler()
    df2 = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns)
    kmeans = KMeansPP(num_clusters=5)
    kmeans.fit(df2.values)
    return df2

class KMeansPP:
    def __init__(self, num_clusters=5):
        self.num_clusters = num_clusters
        self.clusters_ = None
        self.centroids_ = None
        self.labels_ = None
    
    def compute_distance(self, x, centroid):
        return ((x - centroid) ** 2).sum(axis=1)

    def get_closest_cluster(self, x, centroids):
        dist = self.compute_distance(x, centroids)
        return np.argmin(dist)
    
    def compute_sse(self, data, centroids, clusters):
        return self.compute_distance(data, centroids[clusters]).sum() / len(data)

    def get_init_centroid(self, X):
        centroids = []
        centroids.append(X[np.random.randint(X.shape[0]), :])
        for k in range(self.num_clusters - 1):
            dist = []
            for i in range(X.shape[0]):
                point = X[i]
                point = point[None, :]
                d = None
                for j in range(len(centroids)):
                    temp_dist = self.compute_distance(point, centroids[j])
                    if d is None: d = temp_dist
                    d = min(d, temp_dist)
                dist.append(d)
                ## select  point with maximum distance as our next centroid
            dist = np.array(dist)
            next_centroid = X[np.argmax(dist), :]
            centroids.append(next_centroid)
        return np.array(centroids)

    def fit(self, X, max_iterations=1000):
        # k-means
        #centroids = X[random.sample(range(X.shape[0]), self.num_clusters)]
        centroids = self.get_init_centroid(X)
        self.clusters_ = np.zeros(X.shape[0], dtype=np.int32)
        n = 1
        diff = 1
        while n < max_iterations and diff != 0:
            n = n + 1
            for i in range(len(X)):
                x = X[i]
                x = x[None, :]
                # Get the closest cluster
                self.clusters_[i] = self.get_closest_cluster(x, centroids)
            # Loop over centroids and compute the new ones.
            old_centroids = np.copy(centroids)
            for c in range(len(centroids)):
                cluster_data = X[self.clusters_ == c]
                # Compute the average of cluster members to compute new centroid
                new_centroid = cluster_data.mean(axis=0)
                # assign the new centroid
                centroids[c] = new_centroid
            # Compute the SSE for the iteration
            sse = self.compute_sse(X, centroids, self.clusters_)
            diff = np.count_nonzero(old_centroids - centroids)
        self.centroids = centroids
        self.labels_ = self.clusters_
        return self.centroids

def problem3(data):
    df = data[list(set(data.columns)-set(['country']))]
    kmeanspp = KMeansPP(num_clusters=6)
    kmeanspp.fit(df.values)  
    labels = kmeanspp.labels_
    decomp_df = decompostion(df)
    df = data
    df['kmeans_label'] = labels
    df['pca_dim_1'] = decomp_df['pca_dim_1']
    df['pca_dim_2'] = decomp_df['pca_dim_2']
    return df
    
def decompostion(df):
    x = df.values
    pca = PCA(n_components=2, random_state=265)
    reduced_x = pca.fit_transform(x)
    reduced_df = pd.DataFrame(reduced_x, columns=['pca_dim_1', 'pca_dim_2'])
    return reduced_df


def problem4(data):
    mpl.rcParams['figure.dpi'] = 600
    plt.rcParams['figure.figsize'] = (20.0, 10.0)
    plt.rcParams['font.family'] = "serif"
    plt.rcParams['font.size'] = 10

    df = data
    pal = sns.color_palette("Paired")[:len(set(df['kmeans_label']))]
    p1 = sns.scatterplot(x="pca_dim_1", y='pca_dim_2', hue='kmeans_label', palette = pal, data=df, s=250, alpha=0.7, legend=False)

    #For each point, we add a text inside the bubble
    for line in range(0,df.shape[0]):
        p1.text(df.pca_dim_1[line], df.pca_dim_2[line], df.country[line], horizontalalignment='left', size='medium', color='black', weight='semibold')
        plt.suptitle('Two-Dimensional Map of Countries (PCA)', fontsize=36)
        plt.xlabel('PCA - Dimension 1', fontsize=24)
        plt.ylabel('PCA - Dimension 2', fontsize=24)

    for i in df.kmeans_label.unique():
        # get the convex hull
        points = df[df.kmeans_label == i][['pca_dim_1', 'pca_dim_2']].values
        hull = ConvexHull(points)
        x_hull = np.append(points[hull.vertices,0],
                       points[hull.vertices,0][0])
        y_hull = np.append(points[hull.vertices,1],
                       points[hull.vertices,1][0])
    
        # interpolate
        dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
        dist_along = np.concatenate(([0], dist.cumsum()))
        spline, u = interpolate.splprep([x_hull, y_hull],u=dist_along, s=0)
        interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
        interp_x, interp_y = interpolate.splev(interp_d, spline)
        # plot shape
        plt.fill(interp_x, interp_y, '--', c=pal[i], alpha=0.2)
    

    plt.grid()
    plt.show()
    



In [181]:
main()

5) Responses:

a. It might be hard to tell, but Myanmar and Nepal are very similar (if you can read it). There are a bunch of countries in clustered together in the top left cluster. DR Congo, Yemen, Burundi, Cameroon, Sudan are clustered in the dark blue cluster. These countries are clustered together because they share similar characteristics. To clarify, the variables that are compared among the countries are very similar in terms of value. For example, Sudan and Yemen might have very close indexes in terms of public health and other variables. 

b. When I ran my kmeans++ algorithm more than once (3 times to be specific), My PCA map does change. However, when I run the scikit kmeans++ algorithm, the map does not change. Nonetheless, I am assuming that every time you modify the data or re run the kmeans++ alogithm, the map changes. 

c. To an extent it is an accurate clustering of countries. If you use the scikit's kmeans++ function, then it very much acccurate because the cluster groups do not change. It is still accurate because the location of the country on the PCA map does not change no matter what cluster group it is in. I would not say the results change if we have greater variables because there are so many variables already in effect. If there were fewer variables, then the addition of another would have a significant effect. 

d. I think the PCA actually helped the kmeans clustering results. According to K-means Clustering via Principal Component Analysis by Chris Ding and Xiaofeng He, the intuition is that PCA seeks to represent all 𝑛 data vectors as linear combinations of a small number of eigenvectors, and does it to minimize the mean-squared reconstruction error. In contrast, K-means seeks to represent all 𝑛 data vectors via small number of cluster centroids.